In [1]:
from utils import *

# Code Assistant
This agent is responsible for creating and executing python files and connecting to live data.

In [2]:
class ExecutePyFile(OpenAISchema):
    """Run existing python file from local disc."""
    file_name: str = Field(
        ..., description="The path to the .py file to be executed."
    )

    def run(self):
        """Executes a Python script at the given file path and captures its output and errors."""
        try:
            result = subprocess.run(
              ['python3', self.file_name],
              text=True,
              capture_output=True,
              check=True
            )
            return result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred: {e.stderr}"

class File(OpenAISchema):
    """
    Python file with an appropriate name, containing code that can be saved and executed locally at a later time. This environment has access to all standard Python packages and the internet.
    """
    chain_of_thought: str = Field(...,
        description="Think step by step to determine the correct actions that are needed to be taken in order to complete the task.")
    file_name: str = Field(
        ..., description="The name of the file including the extension"
    )
    body: str = Field(..., description="Correct contents of a file")

    def run(self):
        with open(self.file_name, "w") as f:
            f.write(self.body)

        return "File written to " + self.file_name

In [3]:
code_assistant_funcs = [File, ExecutePyFile]
prompt = '''As a top-tier programming AI, your expertise lies in crafting precise Python scripts. Whether it involves naming files or incorporating the right imports, your goal is to seamlessly address user requests. Prioritize code execution before responding to ensure accuracy and reliability in your solutions.
            Additionally, you possess proficiency in handling log data with various features. 
            The available fields in the log are as follows:
                        - `reg_time`: Timestamp of the impression.
                        - `uid`: Unique identifier for the impression.
                        - `fc_imp_chk`: Number of preceding impressions.
                        - `fc_time_chk`: Time since the last impression.
                        - `utmtr`: View time in the user's timezone.
                        - `mm_dma`: Designated Market Area.
                        - `osName`: Operating System.
                        - `model`: Device model.
                        - `hardware`: Device type.
                        - `site_id`: ID of the site where the impression occurred.
                        - `tag`: Type of event (view-through or click-through).The letter <v> at the beginning of certain events indicates that the event occurred without a click, which is referred to as a view-through. Otherwise, we have a click-through event; the fclick event represents the first click (used for calculating CTR).'''

code_assistant = client.beta.assistants.create(
    name='Code Assistant Agent',
    instructions=prompt,
    model="gpt-4-1106-preview",
    tools=[{"type": "function", "function": File.openai_schema},
         {"type": "function", "function": ExecutePyFile.openai_schema},
        #  {"type": "function", "function": SendMessage1.openai_schema},
         ],
)

# User Proxy
This agent facilitates the conversation between the user and other agents, ensuring successful completion of the task.

In [5]:
agents_and_threads = {
    "code_assistant": {
        "agent": code_assistant,
        "thread": None,
        "funcs": code_assistant_funcs
    }
}

class SendMessage(OpenAISchema):
    """Send messages to other specialized agents in this group chat."""
    recepient:Literal['code_assistant'] = Field(..., description="code_assistant is a world class programming AI capable of executing python code.")
    message: str = Field(...,
        description="Specify the task required for the recipient agent to complete. Focus instead on clarifying what the task entails, rather than providing detailed instructions.")

    def run(self):
        recepient = agents_and_threads[self.recepient]
        # if there is no thread between user proxy and this agent, create one
        if not recepient["thread"]:
            recepient["thread"] = client.beta.threads.create()
        message = get_completion(message=self.message, **recepient)

        return message

In [6]:
user_proxy_tools = [SendMessage]

user_proxy = client.beta.assistants.create(
  name='User Proxy Agent',
  instructions="""As a user proxy agent, your responsibility is to streamline the dialogue between the user and specialized agents within this group chat.
Your duty is to articulate user requests accurately to the relevant agents and maintain ongoing communication with them to guarantee the user's task is carried out to completion.
Please do not respond to the user until the task is complete, an error has been reported by the relevant agent, or you are certain of your response.""",
  model="gpt-4-1106-preview",
  tools=[
      {"type": "function", "function": SendMessage.openai_schema},
  ],
)

In [7]:
# Using the data provided in `./data/data.csv`, analyze and identify the key factors that impact ad clicks. Rank the most influential predictors for determining the likelihood of someone clicking on online ads, where values less than 0 indicate a no-click impact, and values greater than 0 signify an influence on the model predicting a click. Please note that the 'tag' column should not be considered as a feature, as it serves as the label for the remaining features.
# Additionally, visualize the feature importance. Present this information in an ordered plot, arranging features in increasing order of importance. After save the visualization with name `feature_importance`.

In [8]:
thread = client.beta.threads.create()
while True:
    user_message = input("User: ")
    print(f"User input: {user_message}")
    message = get_completion(user_message, user_proxy, user_proxy_tools, thread)
    
    wprint(f"\033[34m{user_proxy.name}: ", message,'\033[0m')    

User input: Using the data provided in `./data/data.csv`, analyze and identify the key factors that impact ad clicks. Rank the most influential predictors for determining the likelihood of someone clicking on online ads, where values less than 0 indicate a no-click impact, and values greater than 0 signify an influence on the model predicting a click. Please note that the 'tag' column should not be considered as a feature, as it serves as the label for the remaining features.
Function(arguments='{"recepient":"code_assistant","message":"Plea
se load the data from `./data/data.csv` and conduct an analysis to
identify the key factors that impact ad clicks. Exclude the \'tag\'
column from the features as it serves as the label. Then, using an
appropriate model, rank the predictors by their importance in
determining the likelihood of a click. The ranking should indicate
whether each predictor has a no-click impact (values less than 0) or
an influence on predicting a click (values greater th

KeyboardInterrupt: 